In [1]:
%matplotlib inline
import fceulib
from fceulib import VectorBytes
import numpy as np
import time
import matplotlib.pyplot as plt
from fceu_help import *
from math import *
import pickle

In [2]:

START = 0x08
RUN = 0x02
JUMP = 0x01

#inputVec = fceulib.readInputs('videos/algar-batman.fm2')
#emu = fceulib.runGame('batman.nes')




In [11]:
inputVec = fceulib.readInputs('videos/happylee4-smb-warpless.fm2')
emu = fceulib.runGame('mario.nes')
image_prefix = 'images/mario'

#inputVec = fceulib.readInputs('videos/Contra by 3D.fm2')
#emu = fceulib.runGame('contra.nes')

start = VectorBytes()
emu.save(start)


peekevery = 1
emu.load(start)
display = False
img_buffer = VectorBytes()
start = 270
end = float('inf')
colorized2id = {}
id2colorized = {}
#outfile = open('test.txt','wb')
for timestep, inp in enumerate(inputVec):
    emu.stepFull(inp,0x0)
    if not (timestep % peekevery == 0) or timestep < start  :
        continue
    if timestep > end:
        break
    if timestep % 100 == 0:
        print (100*float(timestep)/float(len(inputVec)))
    sprite_list,colorized_sprites = get_all_sprites(emu.fc)
    if display:
        outputImage(emu, '{}_{}'.format(image_prefix,timestep),img_buffer)

    for sprite_id,sprite in enumerate(sprite_list):
        if np.sum(colorized_sprites[sprite_id].ravel()) == 0:
            continue
        uniq = tuple(colorized_sprites[sprite_id].ravel())
        if uniq not in colorized2id:
            colorized2id[uniq] = len(colorized2id)
            print len(colorized2id)
            id2colorized[colorized2id[uniq]] = colorized_sprites[sprite_id]
        #outfile.write('{},{},{},{}\n'.format(timestep,colorized2id[uniq],sprite[0],sprite[1]))

#outfile.close()
pickle.dump((colorized2id,id2colorized),open('id2sprites.pkl','wb'))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
0.446401999881
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
0.595202666508
39
40
41
42
43
44
45
46
0.744003333135
0.892803999762
47
48
49
50
1.04160466639
1.19040533302
1.33920599964
1.48800666627
51
52
53
54
55
56
57
1.6368073329
1.78560799952
58
59
60
61
1.93440866615
62
63
2.08320933278
2.2320099994
2.38081066603
2.52961133266
64
65
66
67
2.67841199929
2.82721266591
2.97601333254
3.12481399917
3.27361466579
3.42241533242
3.57121599905
68
69
70
71
72
73
3.72001666567
74
75
76
77
78
79
80
81
82
83
84
85
86
3.8688173323
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
4.01761799893
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
4.16641866556
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186

In [ ]:


#uses normalized pmi as a thresholding mechanism 
# -1 = no co-occurences, 
# 0 = independent 
# 1 = only co-occur
def getFusedPairs(filename,fc,threshold=0.1):
    #Read in data
    with open(filename) as infile:
        data = []
        for line in infile:
            data.append([int(float(d)) for d in line.rstrip().split(',')])
    #get the per timestep data, as well as total counts for each sprite
    timesteps = {}
    pSprite = {}
    for d in data:
        if d[0] not in timesteps:
            timesteps[d[0]] = []
        timesteps[d[0]].append(d[1:])
        if d[1] not in pSprite:
            pSprite[d[1]] = 0
        pSprite[d[1]] += 1
    spr = get_sprite(0,emu.fc)
    height = spr.shape[0]
    #for each timestep look at all the sprite pairs and see how far apart they are,
    #if they are within width and height of each other, add that as a co-occurence
    
    all_sprites = {}
    for timestep in sorted(timesteps):
        dat = timesteps[timestep]
        for ii in range(len(dat)):
            for jj in range(len(dat)):
                if ii != jj:
                    id1 = dat[ii][0]
                    id2 = dat[jj][0]
                    dx = dat[ii][1]-dat[jj][1]
                    dy = dat[ii][2]-dat[jj][2]
                    
                    if abs(dx) > 8 or abs(dy) > height:
                        continue
                    if id1 not in all_sprites:
                        all_sprites[id1] = {}
                    if id2 not in all_sprites[id1]:
                        all_sprites[id1][id2] = 0
                    all_sprites[id1][id2] += 1
    
    #now we have all of the pieces and can calculate the pmi for each pair of sprites
    accepted = set()
    for sprite in all_sprites:
        total = 0
        dat = []
        px = float(pSprite[sprite])/float(len(timesteps))

        for other in all_sprites[sprite]:
            py = float(pSprite[other])/float(len(timesteps))
            pxy = float(all_sprites[sprite][other])/float(len(timesteps))
            d = log(px*py)/log(pxy)-1
            if d > threshold:
                s = sprite
                o = other
                if sprite > other:
                    o = sprite
                    s = other
                accepted.add((s,o))
    return accepted
getFusedPairs('test.txt',emu.fc)

In [ ]:
help(fceulib)